In [1]:
from Kyber import *
from Kyber_failure import *
from proba_util import *

In [2]:
ps_light = KyberParameterSet(256, 2, 3, 3, 3329, 2**12, 2**10, 2**4, ke_ct=2)
ps_recommended = KyberParameterSet(256, 3, 2, 2, 3329, 2**12, 2**10, 2**4)
ps_paranoid = KyberParameterSet(256, 4, 2, 2, 3329, 2**12, 2**11, 2**5)

In [3]:
def hammer_law(A, bit_order, pos=True):
    B = {}
    for x in A:
        if (x < 0 and pos) or (x >=0 and not pos):
            continue
        if pos:
            y = x + 2**bit_order
        else:
            y = x - 2**bit_order
        B[y] = B.get(y, 0) + A[x]
    return B

In [36]:
def tail_probability_mod(D, t):
    s = 0
    ma = max( [abs(x) for x in D.keys()] ) 
    if t >= ma:
        return 0
    for i in reversed(range(int(ceil(t)), ma)):  # Summing in reverse for better numerical precision (assuming tails are decreasing)
        s += D.get(i, 0) + D.get(-i, 0)
    return s

In [68]:
def default_filter(A, threshold=2):
    B = {}
    for x in A:
        if x < threshold:
            continue
        B[x] = A[x]
    S = sum(B.values())
    for x in B:
        B[x] = B[x]/S
    return B

In [70]:
def hammer_error_distribution(ps, hammer_bits=10, bit_order=5, honest=True, threshold=2):
    """
    modification of p2_cyclotomic_final_error_distribution in Kyber_failure.py
    """
    
    # s secret key
    chis = build_centered_binomial_law(ps.ks)           # LWE error law for the key
    
    # hammer s
    chis_h = hammer_law(chis, bit_order)
    
    # e_1 and e_2
    chie = build_centered_binomial_law(ps.ke_ct)        # LWE error law for the ciphertext
    
    # e_1 adversarial
    chie_a = default_filter(chie, threshold)
    
    # e in public key, same as s
    chie_pk = build_centered_binomial_law(ps.ke)
    
    # c_t
    Rk = build_mod_switching_error_law(ps.q, ps.rqk)    # Rounding error public key
    
    # c_u
    Rc = build_mod_switching_error_law(ps.q, ps.rqc)    # rounding error first ciphertext
    
    # e + c_t
    chiRs = law_convolution(chis, Rk)                   # LWE+Rounding error key
    
    # e_1 + c_u
    chiRe = law_convolution(chie, Rc)                   # LWE + rounding error ciphertext
    
    # e_1_a + c_u
    chiRe_a = law_convolution(chie_a, Rc)
    

    # ( e + c_t) * r
    B1 = law_product(chie_pk, chiRs)                       # (LWE+Rounding error) * LWE (as in a E*S product)
    
    # s * (e_1 + c_u )
    B2 = law_product(chis, chiRe)
    
    if honest:
        # s_h * (e_1 + c_u)
        B3 = law_product(chis_h, chiRe)
    else:
        # s_h * (e_1_a + c_u)
        B3 = law_product(chis_h, chiRe_a)
    
    
    C1 = iter_law_convolution(B1, ps.m * ps.n)
    
    C2 = iter_law_convolution(B2, ps.m * ps.n - hammer_bits)
    
    C3 = iter_law_convolution(B3, hammer_bits)

    C = law_convolution(C1, C2)
    C = law_convolution(C, C3)
    
    # c_v
    R2 = build_mod_switching_error_law(ps.q, ps.rq2)    # Rounding2 (in the ciphertext mask part)
    
    # c_v + e_2
    F = law_convolution(R2, chie)                       # LWE+Rounding2 error
    
    D = law_convolution(C, F)                           # Final error
    return D

In [49]:
def hammer_error_probability(ps, **kwargs):
    F = hammer_error_distribution(ps, **kwargs)
    prob = tail_probability_mod(F, ps.q/4)
    return prob

In [79]:
log(hammer_error_probability(ps_recommended, hammer_bits=3, bit_order=6, honest=True), 2)*256

-24.507338392395848

In [71]:
log(hammer_error_probability(ps_recommended, hammer_bits=10, bit_order=4, honest=False, threshold=1), 2)

-54.32811546535331

In [76]:
log(hammer_error_probability(ps_recommended, hammer_bits=10, bit_order=5, honest=False, threshold=2), 2)

-9.157078973587836

In [82]:
if 0.1:
    print(True)

True


In [1]:
from functools import *

In [4]:
reduce(lambda x, y: x+y, [1,2])

3

In [9]:
def f(a):
    a[0] = 100
    return a

In [10]:
b = [1,2,3]

In [11]:
f(b)

[100, 2, 3]

In [12]:
b

[100, 2, 3]